# Recommendation System

## Задание 1

Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). Оставь в датасете только фильмы, которые вышли в "релиз".\
Выведи количество фильмов, оставшихся после фильтрации.

In [1]:
import pandas as pd

# Загрузка данных из CSV файлов
credits_df = pd.read_csv('../datasets/tmdb_5000_credits.csv')
movies_df = pd.read_csv('../datasets/tmdb_5000_movies.csv')

# Переименование колонки "movie_id" в "id" для последующего объединения
credits_df.rename(columns={'movie_id': 'id'}, inplace=True)

# Объединение данных по колонке "id"
merged_df = pd.merge(movies_df, credits_df, on='id')

# Фильтрация фильмов, которые вышли в "релиз"
filtered_df = merged_df[merged_df['status'] == 'Released']

# Вывод количества фильмов после фильтрации
num_movies_remaining = len(filtered_df)
print("Количество фильмов после фильтрации:", num_movies_remaining)


Количество фильмов после фильтрации: 4795


## Задание 2

Самый наивный подход к рекомендации фильмов - рекомендовать фильмы с лучшими оценками пользователей. Фильмы, которые пользуются большей популярностью и признанием критиков, с большей вероятностью понравятся среднему зрителю.

Для справедливой оценки фильмов возьмем текущую рейтинговую систему IMDB (weighted rating (WR)), которая рассчитывается по формуле:
$$WR = \frac{v}{v + m} ⋅ R + \frac{m}{v + m} ⋅ C$$ 
$v$ - количество голосов \
$m$ - количество голосов для включения в финальную таблицу \
$R$ - средняя оценка \
$C$ - средняя оценка всех фильмов 

Имплементируй функцию `weighted_rating`. С её помощью расcчитай рейтинг для каждого фильма и сохрани его в колонку `simple_score`.\
Выведи топ-5 фильмов по получившемуся рейтингу.
> В качестве параметра $m$ выбери 95-й квантиль количества голосов.

In [2]:
def weighted_rating(v, m, R, C):
    x = (v / (v + m)) * R + (m / (v + m)) * C
    return  x

In [3]:
import pandas as pd
import numpy as np

# Расчет 95-го квантиля количества голосов (m)
m = filtered_df['vote_count'].quantile(0.95)

# Средняя оценка всех фильмов (C)
C = filtered_df['vote_average'].mean()

# Расчет рейтинга для каждого фильма
filtered_df.loc[:, 'simple_score'] = filtered_df.apply(lambda row: weighted_rating(row['vote_count'], m, row['vote_average'], C), axis=1)

# Вывод топ-5 фильмов по рейтингу (взвешенному рейтингу)
top_5_movies = filtered_df.sort_values('simple_score', ascending=False).head(5)

print(top_5_movies[['original_title', 'vote_count', 'vote_average', 'simple_score']])


                original_title  vote_count  vote_average  simple_score
1881  The Shawshank Redemption        8205           8.5      7.849025
65             The Dark Knight       12002           8.2      7.773990
662                 Fight Club        9413           8.3      7.761012
96                   Inception       13752           8.1      7.736496
3232              Pulp Fiction        8428           8.3      7.714726


/usr/lib/python3/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


## Задание 3

Такой подход к рекомендациям очень наивен, так как не учитывает информацию о самом фильме (жанр, режиссер, описание, актеры и т.п). \
**Content Based Filtering** (Фильтрация на основе содержания) - тип рекомендательной системы, которая предлагает пользователям похожие элементы на основе конкретного элемента. Общая идея этих рекомендательных систем заключается в том, что если человеку понравился определенный товар, то ему понравится и похожий на него товар.

<center><img src="../misc/images/content.png" alt= “” width="300" height="500">

Реализуем алгоритм рекомендации на основе описания фильма. Для это требуется провести предобработку текста:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Расcчитай [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильма

Выведи размер получившейся матрицы Tf-Idf

> Для [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) используйте параметры по умолчанию 

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text

# Замена NaN в описании фильма на пустой символ ''
filtered_df.loc[:, 'overview'] = filtered_df['overview'].fillna('')

# Удаление английских стоп слов
stop_words = list(text.ENGLISH_STOP_WORDS)
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)

# Расчет Tf-Idf для описания фильма
tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_df['overview'])

# Вывод размера получившейся матрицы Tf-Idf
print("Размер матрицы Tf-Idf:", tfidf_matrix.shape)


Размер матрицы Tf-Idf: (4795, 20970)


/usr/lib/python3/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


## Задание 4

Теперь тебе необходимо вычислить показатель сходства между описаниями фильмов. Используем косинусное расстояние, оно рассчитывается по формуле:
$$cos(Θ) = \frac{A ⋅ B}{∥A∥ ∥B∥} = \frac{ Σ_{i=1}^{n} A_i ⋅ B_i } { \sqrt{Σ_{i=1}^{n}A_{i}^{2}} ⋅ {\sqrt{Σ_{i=1}^{n}B_{i}^{2}}}}$$
Но поскольку мы использовали векторизатор TF-IDF на предыдущем шаге, достаточно вычислить скалярное произведение, которое и даст оценку косинусного сходства. Рассчитать его можно через [linear_kernel](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.linear_kernel.html). Результат сохрани в переменную `cosine_sim`.

Выведи размер получившейся матрицы `cosine_sim`.

In [5]:
from sklearn.metrics.pairwise import linear_kernel

# Рассчитываем косинусное сходство между описаниями фильмов
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Вывод размера получившейся матрицы cosine_sim
print("Размер матрицы cosine_sim:", cosine_sim.shape)


Размер матрицы cosine_sim: (4795, 4795)


## Задание 5

Напиши функцию `get_recommendations`. На вход она принимает:
* `movies_dataset` - датасет фильмов
* `title` - название фильма, для которого мы будем искать похожие
* `cosine_sim` - матрица расстояний между описаниями
* `top_k` - топ-k cхожих фильмов

Возвращает top_k названий фильмов, описание которых похоже на выбранный фильм.\
Выведи топ-5 фильмов для `title='Saving Private Ryan'`

In [6]:
def get_recommendations(movies_dataset, title, cosine_sim, top_k=10):
    # Получаем индекс фильма по его названию
    idx = movies_dataset[movies_dataset['original_title'] == title].index[0]

    # Получаем показатели сходства для фильма с выбранным индексом
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Сортируем фильмы по показателям сходства в убывающем порядке
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Индексы топ-K фильмов с наибольшими показателями сходства
    top_indices = [i[0] for i in sim_scores[1:top_k+1]]

    # Возвращаем топ-K названий фильмов
    return movies_dataset['original_title'].iloc[top_indices] 

In [7]:
recommendations = get_recommendations(movies_dataset=filtered_df, title='Saving Private Ryan', cosine_sim=cosine_sim)
print(recommendations)

787         The Great Raid
586      The Monuments Men
290      The Expendables 2
4168             Abandoned
3516             The Train
1525        Apocalypse Now
2430             Evil Dead
3603     Lone Wolf McQuade
3586    As Above, So Below
755     Star Trek: Nemesis
Name: original_title, dtype: object


## Задание 6

Еще один подход к построению рекомендательной системы - подход на основе сходства между пользователями. Этот подход называется **Collaborative Filtering** (Коллаборативная фильтрация).
<center><img src="../misc/images/all.png" alt= “” width="600" height="700"></center>
Коллаборативная фильтрация - это тип рекомендательной системы, которая использует поведение и предпочтения похожих пользователей для рекомендации товаров или продуктов конкретному пользователю. Система собирает данные о прошлом поведении пользователей, такие как покупки, рейтинги и отзывы, и анализирует их для выявления закономерностей и сходства между пользователями. На основе этих закономерностей система рекомендует товары, которые понравились или были приобретены другими такими же пользователями в прошлом.

Для реализации Коллаборативной фильтрации нам потребуются оценки пользователей [ratings](../datasets/ratings.csv).

>userId - id пользователя \
movieId - id фильма \
rating - оценка фильма (от 0 до 5)\
timestamp - время оценки


Воспользуйся библиотекой [surprise](https://surpriselib.com/) для обучения модели оценки рейтинга фильма [SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD). Выведи средние значения 'RMSE', 'MAE' на кросс-валидации с параметрами `cv=5`.

In [13]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.matrix_factorization import SVD

# Загрузка данных оценок пользователей из файла 'ratings.csv'
ratings = pd.read_csv('../datasets/ratings.csv')

# Загружаем данные оценок пользователей
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Создаем модель SVD
svd_model = SVD()

# Выполняем кросс-валидацию с параметрами cv=5
results = cross_validate(svd_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Выводим средние значения RMSE и MAE
print("Среднее RMSE на кросс-валидации:", results['test_rmse'].mean())
print("Среднее MAE на кросс-валидации:", results['test_mae'].mean())


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9073  0.8876  0.8985  0.8955  0.8997  0.8977  0.0064  
MAE (testset)     0.6973  0.6852  0.6902  0.6914  0.6933  0.6915  0.0039  
Fit time          0.73    0.71    0.71    0.70    0.70    0.71    0.01    
Test time         0.07    0.06    0.11    0.06    0.11    0.08    0.02    
Среднее RMSE на кросс-валидации: 0.8977438494228908
Среднее MAE на кросс-валидации: 0.6914625715693313
